# Approximate solvers for the Euler equations of gas dynamics

In this chapter we discuss approximate solvers for the one-dimensional Euler equations:

\begin{align}
    \rho_t + (\rho u)_x & = 0 \\
    (\rho u)_t + (\rho u^2 + p)_x & = 0 \\
    E_t + ((E+p)u)_x & = 0.
\end{align}

As in [Euler_equations.ipynb](Euler_equations.ipynb), we focus on the case of an ideal gas, for which the total energy is given by

\begin{align} \label{EA:EOS}
    E = \frac{p}{\gamma-1} + \frac{1}{2}\rho u^2.
\end{align}

## Roe solver

We first derive a Roe solver for the Euler equations, following the same approach as in [shallow_water_approximate_solvers.ipynb](shallow_water_approximate_solvers.ipynb).  Namely, we assume that $\hat{A} = f'(\hat{q})$ for some average state $\hat{q}$, and impose the condition of conservation:

\begin{align} \label{EA:cons}
    f'(\hat{q}) (q_r - q_l) & = f(q_r) - f(q_l).
\end{align}

We will need the following quantities:

\begin{align}
q & = \begin{pmatrix} \rho \\ \rho u \\ E \end{pmatrix}, & f(q) & = \begin{pmatrix} \rho u \\ \rho u^2 + p \\ H u \rho \end{pmatrix}, \\
f'(\hat{q}) & = \begin{pmatrix} 
                0 & 1 & 0 \\ 
                \frac{\gamma-3}{2}\hat{u}^2 & (3-\gamma)\hat{u} & \gamma-1 \\
                \frac{\gamma-1}{2}\hat{u}^3 - \hat{u}\hat{H} & \hat{H} - (\gamma-1)\hat{u}^2 & \gamma \hat{u} \end{pmatrix}.
\end{align}

Here $H = \frac{E+p}{\rho}$ is the enthalpy.  We have rewritten most expressions involving $E$ in terms of $H$ because it simplifies the derivation that follows.  We now solve \eqref{EA:cons} to find $\hat{u}$ and $\hat{H}$.  It turns out that, for the case of a polytropic ideal gas, the average density $\hat{\rho}$ plays no role in the Roe solver.

The first equation of \eqref{EA:cons} is an identity, satisfied independently of our choice of $\hat{q}$.  The second equation is (using \eqref{EA:EOS})

\begin{align}
    \frac{\gamma-3}{2}\hat{u}^2 (\rho_r - \rho_l) + (3-\gamma)\hat{u}(\rho_r u_r - \rho_l u_l) + (\gamma-1)\left( \frac{p_r-p_l}{\gamma-1} + \frac{1}{2}(\rho_r u_r^2 - \rho_l u_l^2) \right) & = \rho_r u_r^2 - \rho_l u_l^2 + p_r - p_l,
\end{align}

which simplifies to a quadratic equation for $\hat{u}$:

\begin{align} \label{EA:u_quadratic}
    (\rho_r - \rho_l)\hat{u}^2 - 2(\rho_r u_r - \rho_l u_l) \hat{u} + (\rho_r u_r^2 - \rho_l u_l^2) & = 0,
\end{align}

with roots

\begin{align}
    \hat{u}_\pm & = \frac{\rho_r u_r - \rho_l u_l \mp \sqrt{\rho_r \rho_l} (u_l - u_r)}{\rho_r - \rho_l} = \frac{\sqrt{\rho_r} u_r \pm \sqrt{\rho_l} u_l}{\sqrt{\rho_r}\pm\sqrt{\rho_l}}
\end{align}

Notice that this is identical to the Roe average of the velocity for the shallow water equations, if we replace the density $\rho$ with depth $h$.  As before, we choose the root $u_+$ since it is well defined for all values of $\rho_r, \rho_l$.

Next we find $\hat{H}$ by solving the last equation of \eqref{EA:cons}, which reads  
\begin{align}
    \left( \frac{\gamma-1}{2}\hat{u}^3 - \hat{u}\hat{H} \right)(\rho_r - \rho_l) + \left( \hat{H} - (\gamma-1)\hat{u}^2 \right)(\rho_r u_r - \rho_l u_l) + \gamma \hat{u}(E_r - E_l) & = H_r u_r \rho_r - H_l u_l \rho_l.
\end{align}  
We can simplify this using the equality $\gamma E = \rho H + \frac{\gamma-1}{2}\rho u^2$ and solve for $\hat{H}$ to find  
\begin{align}
    \hat{H} & = \frac{\rho_r H_r (u_r - \hat{u}_+) - \rho_l H_l (u_l - \hat{u}_+)}{\rho_r u_r - \rho_l u_l - \hat{u}_\pm(\rho_r -\rho_l)} \\
    & = \frac{\rho_r H_r (u_r - \hat{u}_+) - \rho_l H_l (u_l - \hat{u}_+)}{\pm\sqrt{\rho_r \rho_l}(u_r-u_l)} \\
    & = \frac{\rho_r H_r - \rho_l H_l \mp\sqrt{\rho_r \rho_l}(H_r - H_l)}{\rho_r - \rho_l} \\
    & = \frac{\sqrt{\rho_r}H_r \pm \sqrt{\rho_l} H_l}{\sqrt{\rho_r}\pm\sqrt{\rho_l}}.
\end{align}  
Once more, we take the plus sign in the final expression for $\hat{H}$.

To implement the Roe solver, we also need the eigenvalues and eigenvectors of the averaged flux Jacobian $f'(\hat{q})$.  These are just the eigenvalues of the true Jacobian, evaluated at the averaged state:
\begin{align}
    \lambda_1 & = \hat{u} - \hat{c}, & \lambda_2 & = \hat{u} & \lambda_3 & = \hat{u} + \hat{c},
\end{align}  
\begin{align}
r_1 & = \begin{bmatrix} 1 \\ \hat{u}-\hat{c} \\ \hat{H}-\hat{u}\hat{c}\end{bmatrix} &
r_2 & = \begin{bmatrix} 1 \\ \hat{u} \\ \frac{1}{2}\hat{u}^2 \end{bmatrix} &
r_3 & = \begin{bmatrix} 1 \\ \hat{u}+\hat{c} \\ \hat{H}+\hat{u}\hat{c}\end{bmatrix}.
\end{align}
Here $\hat{c} = \sqrt{(\gamma-1)(\hat{H}-\hat{u}^2/2)}$.

Solving the system of equations
\begin{align}
q_r - q_l & = \sum_{p=1}^3 \wave^p = \sum_{p=1}^3 \alpha_p r_p
\end{align}  
for the wave strengths gives
\begin{align}
    \alpha_2 & = \delta_1 + (\gamma-1)\frac{\hat{u}\delta_2 - \delta_3}{\hat{c}^2} \\
    \alpha_3 & = \frac{\delta_2 + (\hat{c}-\hat{u})\delta_1 - \hat{c}\alpha_2}{2\hat{c}} \\
    \alpha_1 & = \delta_1 - \alpha_2 - \alpha_3,
\end{align}  
where $\delta = q_r - q_l$.  We now have everything we need to implement the Roe solver.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib as mpl
mpl.rcParams['font.size'] = 8
figsize =(8,4)
mpl.rcParams['figure.figsize'] = figsize

import numpy as np
from exact_solvers import Euler
from clawpack import riemann
from utils import riemann_tools
import matplotlib.pyplot as plt
from collections import namedtuple
from ipywidgets import interact
from ipywidgets import widgets
import matplotlib
Primitive_State = namedtuple('State', Euler.primitive_variables)
Conserved_State = namedtuple('State', Euler.conserved_variables)
gamma = 1.4
problem_data = {}
problem_data['gamma'] = gamma
problem_data['gamma1'] = gamma - 1.0

In [ ]:
def roe_averages(q_l, q_r, gamma=1.4):
    rho_sqrt_l = np.sqrt(q_l[0])
    rho_sqrt_r = np.sqrt(q_r[0])
    p_l = (gamma-1.)*(q_l[2]-0.5*(q_l[1]**2)/q_l[0])
    p_r = (gamma-1.)*(q_r[2]-0.5*(q_r[1]**2)/q_r[0])
    denom = rho_sqrt_l + rho_sqrt_r
    u_hat = (q_l[1]/rho_sqrt_l + q_r[1]/rho_sqrt_r)/denom
    H_hat = ((q_l[2]+p_l)/rho_sqrt_l + (q_r[2]+p_r)/rho_sqrt_r)/denom
    c_hat = np.sqrt((gamma-1)*(H_hat-0.5*u_hat**2))
    
    return u_hat, c_hat, H_hat
    
    
def Euler_roe(q_l, q_r, gamma=1.4):
    """
    Approximate Roe solver for the Euler equations.
    """
    
    rho_l = q_l[0]
    rhou_l = q_l[1]
    u_l = rhou_l/rho_l
    rho_r = q_r[0]
    rhou_r = q_r[1]
    u_r = rhou_r/rho_r
    
    u_hat, c_hat, H_hat = roe_averages(q_l, q_r, gamma)
    
    delta = q_r - q_l
    
    s1 = u_hat - c_hat
    s2 = u_hat
    s3 = u_hat + c_hat
    
    alpha2 = (gamma-1.)/c_hat**2 *((H_hat-u_hat**2)*delta[0]+u_hat*delta[1]-delta[2])
    alpha3 = (delta[1] + (c_hat - u_hat)*delta[0] - c_hat*alpha2) / (2.*c_hat)
    alpha1 = delta[0] - alpha2 - alpha3
    
    r1 = np.array([1., u_hat-c_hat, H_hat - u_hat*c_hat])
    r2 = np.array([1., u_hat, 0.5*u_hat**2])
    q_l_star = q_l + alpha1*r1
    q_r_star = q_l_star + alpha2*r2
    
    states = np.column_stack([q_l,q_l_star,q_r_star,q_r])
    speeds = [s1, s2, s3]
    wave_types = ['contact','contact', 'contact']
    
    def reval(xi):
        rho = (xi<s1)*states[0,0] + (s1<=xi)*(xi<s2)*states[0,1] + (s2<=xi)*(xi<s3)*states[0,2] + (s3<=xi)*states[0,3]
        mom = (xi<s1)*states[1,0] + (s1<=xi)*(xi<s2)*states[1,1] + (s2<=xi)*(xi<s3)*states[1,2] + (s3<=xi)*states[1,3]
        E = (xi<s1)*states[2,0] + (s1<=xi)*(xi<s2)*states[2,1] + (s2<=xi)*(xi<s3)*states[2,2] + (s3<=xi)*states[2,3]
        return rho, mom, E
    
    return states, speeds, reval, wave_types

### Examples

Let's compare the Roe approximation to the exact solution.  As a first example, we use the Sod shock tube.

In [ ]:
left_state  = Primitive_State(Density = 3.,
                              Velocity = 0.,
                              Pressure = 3.)
right_state = Primitive_State(Density = 1.,
                              Velocity = 0.,
                              Pressure = 1.)

q_l = np.array(Euler.primitive_to_conservative(*left_state))
q_r = np.array(Euler.primitive_to_conservative(*right_state))

states, speeds, reval, wave_types = Euler_roe(q_l, q_r)
ex_states, ex_speeds, ex_reval, ex_wt = Euler.exact_riemann_solution(q_l,q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wt, wave_types],
                                                 variable_names=Euler.primitive_variables,
                                                 derived_variables=Euler.cons_to_prim)

interact(plot_function, t=widgets.FloatSlider(min=0,max=0.9,step=0.1));

In [ ]:
Euler.phase_plane_plot(q_l,q_r,approx_states=states,cons_inputs=True)

Recall that in the true solution the middle wave is a contact discontinuity and carries only a jump in the density.  The Roe solver, on the other hand, generates a middle wave that carries a jump in all 3 variables (though the pressure jump is quite small in this example).  For a Riemann problem like this one with zero initial velocity on both sides, the Roe average velocity must also be zero, so the middle wave is stationary; this is of course not typically true in the exact solution, even when $u_l=u_r=0$.

Here is a second example.  Experiment with the initial states to explore how the Roe solution compares to the exact solution.

In [ ]:
left_state  = Primitive_State(Density = 0.1,
                              Velocity = 0.,
                              Pressure = 0.1)
right_state = Primitive_State(Density = 1.,
                              Velocity = 1.,
                              Pressure = 1.)

q_l = np.array(Euler.primitive_to_conservative(*left_state))
q_r = np.array(Euler.primitive_to_conservative(*right_state))

states, speeds, reval, wave_types = Euler_roe(q_l, q_r)
ex_states, ex_speeds, ex_reval, ex_wt = Euler.exact_riemann_solution(q_l,q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wt, wave_types],
                                                 variable_names=Euler.primitive_variables,
                                                 derived_variables=Euler.cons_to_prim)

interact(plot_function, t=widgets.FloatSlider(min=0,max=0.9,step=0.1));

In [ ]:
Euler.phase_plane_plot(q_l,q_r,approx_states=states,cons_inputs=True)

### Single-shock solution
Next we demonstrate the exactness property of the Roe solver by applying it to a case where the left and right states are connected by a single shock wave.

In [ ]:
M = 2.  # Mach number of the shock wave
mu = 2*(M**2-1)/(M*(gamma+1.))
right = Primitive_State(Density = 1.,
                        Velocity = 0.,
                        Pressure = 1.)
c_r = np.sqrt(gamma*right.Pressure/right.Density)

rho_l = right.Density * M/(M-mu)
p_l = right.Pressure * ((2*M**2-1)*gamma+1)/(gamma+1)
u_l = mu*c_r

left = Primitive_State(Density = rho_l,
                       Velocity = u_l,
                       Pressure = p_l)

In [ ]:
q_l = np.array(Euler.primitive_to_conservative(*left))
q_r = np.array(Euler.primitive_to_conservative(*right))

states, speeds, reval, wave_types = Euler_roe(q_l, q_r)
ex_states, ex_speeds, ex_reval, ex_wt = Euler.exact_riemann_solution(q_l,q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wt, wave_types],
                                                 variable_names=Euler.primitive_variables,
                                                 derived_variables=Euler.cons_to_prim)

interact(plot_function, t=widgets.FloatSlider(min=0,max=0.9,step=0.1));

In [ ]:
Euler.phase_plane_plot(q_l,q_r,approx_states=states,cons_inputs=True)

It is evident that the solution consists of a single right-going shock.  The exact solution cannot be seen because it coincides exactly with the Roe solution.  The path of the shock in the first plot also cannot be seen since it is plotted under the path of the rightmost Roe solution wave.  The two solutions differ only in the wave speeds predicted for the other two waves, but since these waves have zero strength this makes no difference.

### Transonic rarefactions and an entropy fix
To be written.

## HLLE Solver

Recall that an HLL solver uses only two waves with a constant state between them.  The Euler equations are our first example for which the number of waves in the true solution is larger than the number of waves in the approximate solution.  As one might expect, this leads to noticeable inaccuracy in solutions produced by the solver.

The left-going wave speed is chosen to be the minimum of the Roe speed for the 1-wave and the characterstic speed $\lambda^1$ in the left state $q_\ell$.  The right-going wave speed is chosen to be the maximum of the Roe speed for the 3-wave and the characteristic speed $\lambda^3$ in the right state $q_r$.  Effectively, this means that
\begin{align}
    s_1 & = \min(u_l - c_l, \hat{u}-\hat{c}) \\
    s_2 & = \max(u_r + c_r, \hat{u}+\hat{c})
\end{align}

Recall that once we have chosen these two wave speeds, conservation dictates the value of the intermediate state:  
\begin{align}  \label{SWA:hll_middle_state}
q_m = \frac{f(q_r) - f(q_l) - s_2 q_r + s_1 q_l}{s_1 - s_2}.
\end{align}

In [ ]:
def Euler_hll(q_l, q_r, gamma=1.4):
    """HLL approximate solver for the Euler equations."""
    
    rho_l = q_l[0]
    rhou_l = q_l[1]
    u_l = rhou_l/rho_l
    rho_r = q_r[0]
    rhou_r = q_r[1]
    u_r = rhou_r/rho_r
    E_r = q_r[2]
    E_l = q_l[2]
    
    u_hat, c_hat, H_hat = roe_averages(q_l, q_r, gamma)
    p_r = (gamma-1.) * (E_r - rho_r*u_r**2/2.)
    p_l = (gamma-1.) * (E_l - rho_l*u_l**2/2.)
    H_r = (E_r+p_r) / rho_r
    H_l = (E_l+p_l) / rho_l
    c_r = np.sqrt((gamma-1.)*(H_r-u_r**2/2.))
    c_l = np.sqrt((gamma-1.)*(H_l-u_l**2/2.))
    
    s1 = min(u_l-c_l,u_hat-c_hat)
    s2 = max(u_r+c_r,u_hat+c_hat)
    
    rho_m = (rhou_r - rhou_l - s2*rho_r + s1*rho_l)/(s1-s2)
    rhou_m = (rho_r*u_r**2 - rho_l*u_l**2 + p_r - p_l - s2*rhou_r + s1*rhou_l)/(s1-s2)
    E_m = ( u_r*(E_r+p_r) - u_l*(E_l+p_l) -s2*E_r + s1*E_l)/(s1-s2)
    q_m = np.array([rho_m, rhou_m, E_m])
    
    states = np.column_stack([q_l,q_m,q_r])
    speeds = [s1, s2]
    wave_types = ['contact','contact']
    
    def reval(xi):
        rho  = (xi<s1)*rho_l + (s1<=xi)*(xi<=s2)*rho_m + (s2<xi)*rho_r
        mom  = (xi<s1)*rhou_l + (s1<=xi)*(xi<=s2)*rhou_m + (s2<xi)*rhou_r
        E  = (xi<s1)*E_l + (s1<=xi)*(xi<=s2)*E_m + (s2<xi)*E_r
        return rho, mom, E

    return states, speeds, reval, wave_types

### Examples

In [ ]:
left_state  = Primitive_State(Density = 3.,
                              Velocity = 0.,
                              Pressure = 3.)
right_state = Primitive_State(Density = 1.,
                              Velocity = 0.,
                              Pressure = 1.)

q_l = np.array(Euler.primitive_to_conservative(*left_state))
q_r = np.array(Euler.primitive_to_conservative(*right_state))

states, speeds, reval, wave_types = Euler_hll(q_l, q_r)
ex_states, ex_speeds, ex_reval, ex_wt = Euler.exact_riemann_solution(q_l,q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wt, wave_types],
                                                 variable_names=Euler.primitive_variables,
                                                 derived_variables=Euler.cons_to_prim)

interact(plot_function, t=widgets.FloatSlider(min=0,max=0.9,step=0.1));

In [ ]:
Euler.phase_plane_plot(q_l,q_r,approx_states=states,cons_inputs=True)

### Preservation of positivity
Just as we saw in the case of the shallow water equations, the Roe solver (or any linearized solver) for the Euler equations fails to preserve positivity of the pressure and/or density in some situations.  Here is one example.

In [ ]:
left_state  = Primitive_State(Density = 1.,
                              Velocity = -5.,
                              Pressure = 1.)
right_state = Primitive_State(Density = 1.,
                              Velocity = 1.,
                              Pressure = 1.)


q_l = np.array(Euler.primitive_to_conservative(*left_state))
q_r = np.array(Euler.primitive_to_conservative(*right_state))

states, speeds, reval, wave_types = Euler_roe(q_l, q_r)
ex_states, ex_speeds, ex_reval, ex_wt = Euler.exact_riemann_solution(q_l,q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wt, wave_types],
                                                 variable_names=Euler.primitive_variables,
                                                 derived_variables=Euler.cons_to_prim)

interact(plot_function, t=widgets.FloatSlider(min=0,max=0.9,step=0.1));

As we can see, in this example each Roe solver wave moves much more slowly than the leading edge of the corresponding true rarefaction.  In order to maintain conservation, this implies that the middle Roe state must have lower density than the true middle state.  This leads to a negative density.

The HLL solver, on the other hand, guarantees positivity of the density and pressure.  Since the HLL wave speed in the case of a rarefaction is always the speed of the leading edge of the true rarefaction, and since the HLL solution is conservative, the density in a rarefaction will always be at least as great as that of the true solution.  This can be seen clearly in the example below.

In [ ]:
left_state  = Primitive_State(Density = 1.,
                              Velocity = -10.,
                              Pressure = 1.)
right_state = Primitive_State(Density = 1.,
                              Velocity = 1.,
                              Pressure = 1.)


q_l = np.array(Euler.primitive_to_conservative(*left_state))
q_r = np.array(Euler.primitive_to_conservative(*right_state))

states, speeds, reval, wave_types = Euler_hll(q_l, q_r)
ex_states, ex_speeds, ex_reval, ex_wt = Euler.exact_riemann_solution(q_l,q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wt, wave_types],
                                                 ['Exact','HLL'],
                                                 layout='vertical',
                                                 variable_names=Euler.primitive_variables,
                                                 derived_variables=Euler.cons_to_prim)

interact(plot_function, t=widgets.FloatSlider(min=0,max=0.9,step=0.1));

*I've used a vertical layout in the example above.  Which should we use in general?*

In [ ]:
Euler.phase_plane_plot(q_l,q_r,approx_states=states,cons_inputs=True)

## Comparison of two approximate solvers with the exact solution


In [ ]:
q_l = np.array(Euler.primitive_to_conservative(*left))
q_r = np.array(Euler.primitive_to_conservative(*right))

states, speeds, reval, wave_types = Euler_roe(q_l, q_r)
ex_states, ex_speeds, ex_reval, ex_wt = Euler.exact_riemann_solution(q_l,q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wt, wave_types],
                                                 variable_names=Euler.primitive_variables,
                                                 derived_variables=Euler.cons_to_prim)

interact(plot_function, t=widgets.FloatSlider(min=0,max=0.9,step=0.1));

In [ ]:
Euler.phase_plane_plot(q_l,q_r,approx_states=states,cons_inputs=True)

It is evident that the solution consists of a single right-going shock.  The exact solution cannot be seen because it coincides exactly with the Roe solution.  The path of the shock in the first plot also cannot be seen since it is plotted under the path of the rightmost Roe solution wave.  The two solutions differ only in the wave speeds predicted for the other two waves, but since these waves have zero strength this makes no difference.

### Transonic rarefactions and an entropy fix
To be written.

## HLLE Solver

Recall that an HLL solver uses only two waves with a constant state between them.  The Euler equations are our first example for which the number of waves in the true solution is larger than the number of waves in the approximate solution.  As one might expect, this leads to noticeable inaccuracy in solutions produced by the solver.

The left-going wave speed is chosen to be the minimum of the Roe speed for the 1-wave and the characterstic speed $\lambda^1$ in the left state $q_\ell$.  The right-going wave speed is chosen to be the maximum of the Roe speed for the 3-wave and the characteristic speed $\lambda^3$ in the right state $q_r$.  Effectively, this means that
\begin{align}
    s_1 & = \min(u_l - c_l, \hat{u}-\hat{c}) \\
    s_2 & = \max(u_r + c_r, \hat{u}+\hat{c})
\end{align}

Recall that once we have chosen these two wave speeds, conservation dictates the value of the intermediate state:  
\begin{align}  \label{SWA:hll_middle_state}
q_m = \frac{f(q_r) - f(q_l) - s_2 q_r + s_1 q_l}{s_1 - s_2}.
\end{align}

In [ ]:
def Euler_hll(q_l, q_r, gamma=1.4):
    """HLL approximate solver for the Euler equations."""
    
    rho_l = q_l[0]
    rhou_l = q_l[1]
    u_l = rhou_l/rho_l
    rho_r = q_r[0]
    rhou_r = q_r[1]
    u_r = rhou_r/rho_r
    E_r = q_r[2]
    E_l = q_l[2]
    
    u_hat, c_hat, H_hat = roe_averages(q_l, q_r, gamma)
    p_r = (gamma-1.) * (E_r - rho_r*u_r**2/2.)
    p_l = (gamma-1.) * (E_l - rho_l*u_l**2/2.)
    H_r = (E_r+p_r) / rho_r
    H_l = (E_l+p_l) / rho_l
    c_r = np.sqrt((gamma-1.)*(H_r-u_r**2/2.))
    c_l = np.sqrt((gamma-1.)*(H_l-u_l**2/2.))
    
    s1 = min(u_l-c_l,u_hat-c_hat)
    s2 = max(u_r+c_r,u_hat+c_hat)
    
    rho_m = (rhou_r - rhou_l - s2*rho_r + s1*rho_l)/(s1-s2)
    rhou_m = (rho_r*u_r**2 - rho_l*u_l**2 + p_r - p_l - s2*rhou_r + s1*rhou_l)/(s1-s2)
    E_m = ( u_r*(E_r+p_r) - u_l*(E_l+p_l) -s2*E_r + s1*E_l)/(s1-s2)
    q_m = np.array([rho_m, rhou_m, E_m])
    
    states = np.column_stack([q_l,q_m,q_r])
    speeds = [s1, s2]
    wave_types = ['contact','contact']
    
    def reval(xi):
        rho  = (xi<s1)*rho_l + (s1<=xi)*(xi<=s2)*rho_m + (s2<xi)*rho_r
        mom  = (xi<s1)*rhou_l + (s1<=xi)*(xi<=s2)*rhou_m + (s2<xi)*rhou_r
        E  = (xi<s1)*E_l + (s1<=xi)*(xi<=s2)*E_m + (s2<xi)*E_r
        return rho, mom, E

    return states, speeds, reval, wave_types

### Examples

In [ ]:
left_state  = Primitive_State(Density = 3.,
                              Velocity = 0.,
                              Pressure = 3.)
right_state = Primitive_State(Density = 1.,
                              Velocity = 0.,
                              Pressure = 1.)

q_l = np.array(Euler.primitive_to_conservative(*left_state))
q_r = np.array(Euler.primitive_to_conservative(*right_state))

states, speeds, reval, wave_types = Euler_hll(q_l, q_r)
ex_states, ex_speeds, ex_reval, ex_wt = Euler.exact_riemann_solution(q_l,q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wt, wave_types],
                                                 variable_names=Euler.primitive_variables,
                                                 derived_variables=Euler.cons_to_prim)

interact(plot_function, t=widgets.FloatSlider(min=0,max=0.9,step=0.1));

In [ ]:
Euler.phase_plane_plot(q_l,q_r,approx_states=states,cons_inputs=True)

### Preservation of positivity
Just as we saw in the case of the shallow water equations, the Roe solver (or any linearized solver) for the Euler equations fails to preserve positivity of the pressure and/or density in some situations.  Here is one example.

In [ ]:
left_state  = Primitive_State(Density = 1.,
                              Velocity = -5.,
                              Pressure = 1.)
right_state = Primitive_State(Density = 1.,
                              Velocity = 1.,
                              Pressure = 1.)


q_l = np.array(Euler.primitive_to_conservative(*left_state))
q_r = np.array(Euler.primitive_to_conservative(*right_state))

states, speeds, reval, wave_types = Euler_roe(q_l, q_r)
ex_states, ex_speeds, ex_reval, ex_wt = Euler.exact_riemann_solution(q_l,q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wt, wave_types],
                                                 variable_names=Euler.primitive_variables,
                                                 derived_variables=Euler.cons_to_prim)

interact(plot_function, t=widgets.FloatSlider(min=0,max=0.9,step=0.1));

As we can see, in this example each Roe solver wave moves much more slowly than the leading edge of the corresponding true rarefaction.  In order to maintain conservation, this implies that the middle Roe state must have lower density than the true middle state.  This leads to a negative density.

The HLL solver, on the other hand, guarantees positivity of the density and pressure.  Since the HLL wave speed in the case of a rarefaction is always the speed of the leading edge of the true rarefaction, and since the HLL solution is conservative, the density in a rarefaction will always be at least as great as that of the true solution.  This can be seen clearly in the example below.

In [ ]:
left_state  = Primitive_State(Density = 1.,
                              Velocity = -10.,
                              Pressure = 1.)
right_state = Primitive_State(Density = 1.,
                              Velocity = 1.,
                              Pressure = 1.)


q_l = np.array(Euler.primitive_to_conservative(*left_state))
q_r = np.array(Euler.primitive_to_conservative(*right_state))

states, speeds, reval, wave_types = Euler_hll(q_l, q_r)
ex_states, ex_speeds, ex_reval, ex_wt = Euler.exact_riemann_solution(q_l,q_r)

plot_function = riemann_tools.make_plot_function([ex_states, states],
                                                 [ex_speeds, speeds],
                                                 [ex_reval, reval],
                                                 [ex_wt, wave_types],
                                                 ['Exact','HLL'],
                                                 layout='vertical',
                                                 variable_names=Euler.primitive_variables,
                                                 derived_variables=Euler.cons_to_prim)

interact(plot_function, t=widgets.FloatSlider(min=0,max=0.9,step=0.1));

*I've used a vertical layout in the example above.  Which should we use in general?*

In [ ]:
Euler.phase_plane_plot(q_l,q_r,approx_states=states,cons_inputs=True)

Notice that the Roe solver significantly understimates the shock speed, and even propagates the contact discontinuity in the wrong direction.  Nevertheless, when used as an ingredient in a numerical method, it gives good results.

# Comparison of full discretizations

How do these solvers impact the solution accuracy when used within a finite volume discretization?

We'll test them on the same shock tube problem considered above.

In [ ]:
from clawpack.riemann.euler_with_efix_1D_constants import density, momentum, energy, num_eqn

def setup(q_l, q_r, N=50, riemann_solver='HLL', solver_type='classic'):

    from clawpack import pyclaw
    from clawpack import riemann

    if riemann_solver == 'Roe':
        rs = riemann.euler_1D_py.euler_roe_1D
    elif riemann_solver == 'HLL':
        rs = riemann.euler_1D_py.euler_hll_1D

    if solver_type == 'classic':
        solver = pyclaw.ClawSolver1D(rs)        
    else:
        solver = pyclaw.SharpClawSolver1D(rs)

    solver.kernel_language = 'Python'
    
    solver.bc_lower[0]=pyclaw.BC.extrap
    solver.bc_upper[0]=pyclaw.BC.extrap

    x = pyclaw.Dimension(-1.0,1.0,N,name='x')
    domain = pyclaw.Domain([x])
    state = pyclaw.State(domain,num_eqn)

    gamma = 1.4
    state.problem_data['gamma']= gamma
    state.problem_data['gamma1']= gamma-1.

    state.problem_data['efix'] = False

    xc = state.grid.p_centers[0]
    
    velocity = (xc<=0)*q_l[1] + (xc>0)*q_r[1]
    pressure = (xc<=0)*q_l[2] + (xc>0)*q_r[2]

    state.q[density ,:] = (xc<=0)*q_l[0] + (xc>0)*q_r[0]
    state.q[momentum,:] = velocity * state.q[density,:]
    state.q[energy  ,:] = pressure/(gamma - 1.) + 0.5 * state.q[density,:] * velocity**2

    claw = pyclaw.Controller()
    claw.tfinal = 0.5
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver
    claw.num_output_times = 10
    claw.keep_copy = True
    claw.verbosity=0

    return claw

## Second-order modified Lax-Wendroff algorithm

In [ ]:
q_l = Euler.conservative_to_primitive(1.,0.,1.)
q_r = Euler.conservative_to_primitive(1./8,0.,1./10)
roe = setup(q_l,q_r,N=50,riemann_solver='Roe')
roe.run()
hll = setup(q_l,q_r,riemann_solver='HLL')
hll.run()
fine = setup(q_l,q_r,N=1000,riemann_solver='Roe')
fine.run();
xc = roe.solution.state.grid.p_centers[0]
xc_fine = fine.solution.state.grid.p_centers[0]

def plot_frame(i):
    fig, ax = plt.subplots(figsize=(12,6))
    ax.set_xlim((-1,1)); ax.set_ylim((0,1.1))
    ax.plot(xc_fine,fine.frames[i].q[density,:],'-k',lw=1)
    ax.plot(xc,hll.frames[i].q[density,:],'-ob',lw=2)
    ax.plot(xc,roe.frames[i].q[density,:],'-or',lw=2)
    plt.legend(['Fine','HLL','Roe'],loc='best')
    
interact(plot_frame, i=widgets.IntSlider(min=0, max=10, description='Frame'));

As you might expect, the HLL solver smears the middle wave (contact discontinuity) significantly more than the Roe solver does. Perhaps surprisingly, it captures the shock just as accurately as the Roe solver does.

## High-order WENO + Runge-Kutta

In [ ]:
q_l = Euler.conservative_to_primitive(1.,0.,1.)
q_r = Euler.conservative_to_primitive(1./8,0.,1./10)
roe2 = setup(q_l,q_r,N=50,riemann_solver='Roe',solver_type='sharpclaw')
roe2.run()
hll2 = setup(q_l,q_r,riemann_solver='HLL',solver_type='sharpclaw')
hll2.run()
fine2 = setup(q_l,q_r,N=1000,riemann_solver='Roe',solver_type='sharpclaw')
fine2.run();
xc = roe.solution.state.grid.p_centers[0]
xc_fine = fine.solution.state.grid.p_centers[0]

def plot_frame(i):
    fig, ax = plt.subplots(figsize=(12,6))
    ax.set_xlim((-1,1)); ax.set_ylim((0,1.1))
    ax.plot(xc_fine,fine2.frames[i].q[density,:],'-k',lw=1)
    ax.plot(xc,hll2.frames[i].q[density,:],'-ob',lw=2)
    ax.plot(xc,roe2.frames[i].q[density,:],'-or',lw=2)
    plt.legend(['Fine','HLL','Roe'],loc='best')
    
interact(plot_frame, i=widgets.IntSlider(min=0, max=10, description='Frame'));

With higher-order discretizations, the difference in the Riemann solvers is less significant.